## POS tagging using modified Viterbi

### Data Preparation

In [799]:
#Importing libraries
# import nltk
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.parse import ViterbiParser

In [800]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

<p><span style="color: #800000;"><strong>Let's Split the data</strong></span></p>

In [801]:
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)
print(len(train_set))
print(len(test_set))

3718
196


In [802]:
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95506

In [803]:
tokens = [pair[0] for pair in train_tagged_words]
print(tokens[:10])
len(tokens)

['So', 'far', ',', 'Mrs.', 'Hills', 'has', "n't", 'deemed', 'any', 'cases']


95506

In [804]:
V = set(tokens)
print(len(V))

12019


In [805]:
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

In [806]:
print(T)

{'.', 'X', 'VERB', 'PRON', 'ADJ', 'NOUN', 'CONJ', 'PRT', 'ADP', 'DET', 'NUM', 'ADV'}


In [807]:
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

## Emission Probability

In [808]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

## Transition Probability

In [809]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [810]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [811]:
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [812]:
tags_df

,.,X,VERB,PRON,ADJ,NOUN,CONJ,PRT,ADP,DET,NUM,ADV
.,0.094346,0.027188,0.089575,0.066979,0.044112,0.221912,0.058246,0.002431,0.090745,0.173569,0.079132,0.051674
X,0.162891,0.074984,0.204212,0.056477,0.017230,0.061742,0.010211,0.185386,0.144065,0.054244,0.002872,0.025686
VERB,0.034927,0.218876,0.168504,0.036324,0.064499,0.109826,0.005588,0.031046,0.091198,0.135051,0.022198,0.081962
PRON,0.041319,0.090599,0.486353,0.007961,0.073920,0.208491,0.005307,0.012130,0.023124,0.009477,0.007582,0.033738
ADJ,0.064585,0.020482,0.012223,0.000661,0.065907,0.699538,0.016848,0.010737,0.078130,0.005121,0.020978,0.004790
NOUN,0.240351,0.029094,0.146455,0.004715,0.011952,0.263779,0.042727,0.043896,0.177120,0.013268,0.009357,0.017288
CONJ,0.033906,0.007896,0.157455,0.058523,0.115653,0.351602,0.000464,0.004180,0.053414,0.121691,0.039944,0.055272
PRT,0.042979,0.013123,0.403871,0.018045,0.086286,0.245407,0.002297,0.001969,0.020669,0.098753,0.056102,0.010499
ADP,0.039936,0.034704,0.008436,0.069941,0.104645,0.321730,0.000854,0.001388,0.017085,0.324826,0.063107,0.013348
DET,0.017358,0.045203,0.039537,0.003616,0.206244,0.637657,0.000482,0.000241,0.009523,0.005545,0.022059,0.012536


In [813]:
len(train_tagged_words)

95506

<p><span style="color: #ff0000;"><strong>Build the Vanilla Viterbi Algorithm</strong></span></p>

In [814]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

<h4><span style="color: #ff0000;"><strong>Check on 10 randomly selected sentences</strong></span></h4>

In [815]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(10)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
# test_tagged_words
# test_run
test_tagged_words[:10]

['Some',
 'Democrats',
 'in',
 'Congress',
 'are',
 'warning',
 'that',
 'a',
 'complicated',
 'new']

In [816]:
test_main = [tup[0] for sent in test_set for tup in sent]

In [817]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [818]:
print(tagged_seq[:10])

[('Some', 'DET'), ('Democrats', 'NOUN'), ('in', 'ADP'), ('Congress', 'NOUN'), ('are', 'VERB'), ('warning', 'VERB'), ('that', 'ADP'), ('a', 'DET'), ('complicated', 'ADJ'), ('new', 'ADJ')]


<h2><span style="color: #339966;">Accuracy for the Vanilla viterbi Algorithm</span></h2>

In [819]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy1 = len(check)/len(tagged_seq)
accuracy1*100

95.58232931726907

In [820]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

<h3><span style="color: #ff0000;">Incorrectly tagged words for vanilla algorithm</span></h3>

In [821]:
# tagged_seq
vanilla_list = incorrect_tagged_cases[:10]

In [822]:
wrong_words = []
right_words = []
for i, j in enumerate(zip(tagged_seq, test_run_base)):
#     print([test_run_base[i-1],j])
    if(j[0]!=j[1]):
        wrong_words.append(j[0])
        right_words.append(j[1])

In [823]:
print(wrong_words)

[('closer', 'ADJ'), ('achieving', '.'), ('downward', 'ADV'), ('Left', '.'), ('more', 'ADJ'), ('sound', 'NOUN'), ('altering', '.'), ('Peoria', '.'), ('71,309', '.'), ('Borough', '.'), ('up', 'ADV')]


In [824]:
print(right_words)

[('closer', 'ADV'), ('achieving', 'VERB'), ('downward', 'ADJ'), ('Left', 'VERB'), ('more', 'ADV'), ('sound', 'VERB'), ('altering', 'VERB'), ('Peoria', 'NOUN'), ('71,309', 'NUM'), ('Borough', 'NOUN'), ('up', 'PRT')]


<h1><span style="color: #003366;">Testing on the test file</span></h1>

In [825]:
## Testing on the test file
file = open("Test_sentences.txt", "r") 
words=[]
for line in file:
    words.append(word_tokenize(line.rstrip()))
word_list = [word for sent in words for word in sent] #This will remain for all our future tests

In [826]:
tagged_test1 = Viterbi(word_list)

In [827]:
tagged_test1[:10]

[('Android', '.'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', '.'),
 ('.', '.')]

### Solve the problem of unknown words

# Rule 1: Instead of taking the emission probability as 0, we will try to put 0.00001 as the value.

In [828]:
# Viterbi Heuristic
def Viterbi_modified(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = (word_given_tag(words[key], tag)[0])/(word_given_tag(words[key], tag)[1]) or 0.00001
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [829]:
tagged_seq = Viterbi_modified(test_tagged_words)

In [830]:
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]
accuracy2 = len(check)/len(tagged_seq)
accuracy2*100

97.18875502008032

In [831]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
Viterbi_modified_list = incorrect_tagged_cases

In [832]:
incorrect_tagged_cases

[[('coming', 'VERB'), (('closer', 'ADJ'), ('closer', 'ADV'))],
 [('the', 'DET'), (('stated', 'NOUN'), ('stated', 'VERB'))],
 [('some', 'DET'), (('downward', 'NOUN'), ('downward', 'ADJ'))],
 [('and', 'CONJ'), (('more', 'ADJ'), ('more', 'ADV'))],
 [('bells', 'NOUN'), (('sound', 'NOUN'), ('sound', 'VERB'))],
 [('$', '.'), (('71,309', 'NOUN'), ('71,309', 'NUM'))],
 [('ran', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))]]

<h2><span style="color: #ff0000;">During the last run, we got the accuracy of&nbsp;95.3%</span></h2>

In [833]:
wrong_words = []
right_words = []
for i, j in enumerate(zip(tagged_seq, test_run_base)):
#     print([test_run_base[i-1],j])
    if(j[0]!=j[1]):
        wrong_words.append(j[0])
        right_words.append(j[1])

In [834]:
print(wrong_words)

[('closer', 'ADJ'), ('stated', 'NOUN'), ('downward', 'NOUN'), ('more', 'ADJ'), ('sound', 'NOUN'), ('71,309', 'NOUN'), ('up', 'ADV')]


In [835]:
print(right_words)

[('closer', 'ADV'), ('stated', 'VERB'), ('downward', 'ADJ'), ('more', 'ADV'), ('sound', 'VERB'), ('71,309', 'NUM'), ('up', 'PRT')]


<h4><strong><span style="color: #339966;">Let's look into preventing misclassification of numbers and further optimisations</span></strong></h4>

### Get the list of unknown words

In [836]:
#Unknown words, getting the list
list1 = [i[0] for i in train_tagged_words]
list2 = test_main
unknown = list((set(list2).difference(set(list1))))
print(unknown[:10])

['arrest', 'Peoria', 'cheapest', 'Blue', 'healing', 'maze', 'Power', 'Spence', 'how-to', '*T*-216']


In [837]:
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense
    (r'.*es$', 'VERB'),               # 3rd singular present
    (r'.*ould$', 'VERB'),              # modals
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                    # nouns
]
regexp_tagger = nltk.RegexpTagger(patterns)
bigram = nltk.BigramTagger(train_set,backoff=regexp_tagger)

<h4><strong><span style="color: #339966;">Testing out our bigram tagger</span></strong></h4>

In [838]:
list1 = test_tagged_words
list1.append(test_tagged_words[0])
bigram.tag(list(list1))[0][1]

'DET'

In [840]:
def Viterbi_test(words,unknown=unknown, train_bag = train_tagged_words):
    state = []
    prob = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:                
                transition_p = tags_df.loc[state[-1], tag]
            # compute emission and state probabilities
            emission_p = (word_given_tag(words[key], tag)[0])/(word_given_tag(words[key], tag)[1]) or 0.00001     
            if words[key] in unknown:
                state_probability = transition_p
                flag=True
#                 print(words[key],transition_p)
            else:
                flag=False
                state_probability = emission_p * transition_p
            p.append(state_probability)
        pmax = max(p)
        # getting state for which probability is maximum
        list1 = []
        if float(pmax) < float(0.00001):
#             list1=[words[key-1],words[key]]
            state.append(bigram.tag(words[key])[0][1])
#             print(state)
        else:
            state_max = T[p.index(pmax)]
            state.append(state_max)
        prob.append(pmax)
#     return list(zip(list(zip(words, state)), prob))
    return list(zip(words, state))    

#### Evaluating tagging accuracy for the random 10 sentences

In [842]:
tagged_words = Viterbi_test(test_tagged_words)

In [843]:
# tagged_words

In [844]:
# print(right_words[:10])

In [845]:
check = [i for i, j in zip(tagged_words, test_run_base) if i == j]
accuracy3 = len(check)/len(tagged_words)
accuracy3*100

96.39999999999999

In [846]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
Viterbi_test_list = incorrect_tagged_cases[:10]

In [847]:
wrong_words = []
right_words = []
for i, j in enumerate(zip(tagged_words, test_run_base)):
#     print([test_run_base[i-1],j])
    if(j[0][0]!=j[1]):
        wrong_words.append(j[0])
        right_words.append(j[1])

In [848]:
right_words[:10]

[('Some', 'DET'),
 ('Democrats', 'NOUN'),
 ('in', 'ADP'),
 ('Congress', 'NOUN'),
 ('are', 'VERB'),
 ('warning', 'VERB'),
 ('that', 'ADP'),
 ('a', 'DET'),
 ('complicated', 'ADJ'),
 ('new', 'ADJ')]

In [849]:
wrong_words[:10]

[('Some', 'DET'),
 ('Democrats', 'NOUN'),
 ('in', 'ADP'),
 ('Congress', 'NOUN'),
 ('are', 'VERB'),
 ('warning', 'VERB'),
 ('that', 'ADP'),
 ('a', 'DET'),
 ('complicated', 'ADJ'),
 ('new', 'ADJ')]

<h4><span style="background-color: #ffffff; color: #ff0000;"><strong>Third modification, directly sending the unknown words to the tagger</strong></span></h4>

In [850]:
def Viterbi_mod2(words,unknown_words=unknown, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1] or 0.00001

            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
#         if (re.search(r'^-?[0-9]+(.[0-9]+)?$',word)!=None):
#             state.append('NUM')
        if words[key] in unknown_words:
#             list1=[words[key-1],words[key]]
            state.append(bigram.tag(words[key])[0][1])
        else:
            state_max = T[p.index(pmax)]
            state.append(state_max)
   
    return list(zip(words, state))

In [851]:
tagged_seq = Viterbi_mod2(test_tagged_words)
check = [i for i, j in zip(tagged_words, test_run_base) if i == j]
accuracy4 = len(check)/len(tagged_words)
accuracy4*100

96.39999999999999

In [852]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
Viterbi_mod2_list = incorrect_tagged_cases

# Comparing sample test file output

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [853]:
tagged_test1 = Viterbi(word_list)
tagged_test2 = Viterbi_test(word_list)
tagged_test3 = Viterbi_modified(word_list)
tagged_test4 = Viterbi_mod2(word_list)

# We can see that viterbi_modified and viterbi mod2 are the best models that we have here, however viterbi_modified gives us the best accuracy.

<ul>
<li>
<h4><span style="color: #000080;">List of Modifications:</span></h4>
<ul>
<li><strong><span style="color: #008000;">Viterbi_Modified - Changed the emission probability directly to 0.00001 instead of 0 for particular cases wehen the emisssion probability will be 0 for the unknown words.</span></strong></li>
<li><strong><span style="color: #008000;">Viterbi_test - If the max probability is very low, use the Bigram and the rule based tagger</span></strong></li>
<li><strong><span style="color: #008000;">Viterbi_mod2 - Send the unknown words directly to the bigram and rule based tagger.</span></strong></li>
</ul>
</li>
</ul>

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [854]:
dt1 = pd.DataFrame(tagged_test1,columns=['word','vanilla_tag'])
dt2 = pd.DataFrame(tagged_test3,columns=['word','modified_tag'])
dt3 = pd.DataFrame(tagged_test2,columns=['word','test_tag'])
dt4 = pd.DataFrame(tagged_test4,columns=['word','mod2_tag'])

In [855]:
dt5= pd.merge(dt1,dt2,on='word')
dt6 = pd.merge(dt1,dt3,on='word')
dt7 = pd.merge(dt1,dt4,on='word')

In [856]:
dt5[dt5['vanilla_tag']!=dt5['modified_tag']]

,word,vanilla_tag,modified_tag
0,Android,.,NOUN
1,Android,.,NOUN
2,Android,.,NOUN
3,Android,.,NOUN
41,Google,.,DET
42,Google,.,NOUN
43,Google,.,X
44,Google,.,DET
45,Google,.,NOUN
46,Google,.,X


<ul>
<li>
<h4><span style="color: #ff0000;"><span style="color: #000080;">From the above dataframe:</span></span></h4>
<ul>
<li><span style="color: #ff0000;"><span style="color: #008000;"><strong>Nouns are modified much better.</strong></span></span></li>
<li><span style="color: #ff0000;"><span style="color: #008000;"><strong>63 changes are listed down in the data frame that were incorrectly determined by the vanilla algorithm</strong></span></span></li>
</ul>
</li>
</ul>

In [857]:
dt6[dt6['vanilla_tag']!=dt6['test_tag']]

,word,vanilla_tag,test_tag
0,Android,.,DET
1,Android,.,DET
2,Android,.,DET
3,Android,.,DET
41,Google,.,NOUN
42,Google,.,NOUN
43,Google,.,NOUN
44,Google,.,NOUN
45,Google,.,NOUN
46,Google,.,NOUN


<ul>
<li>
<h4><span style="color: #ff0000;"><span style="color: #000080;">From the above dataframe:</span></span></h4>
<ul>
<li><span style="color: #ff0000;"><span style="color: #008000;"><strong>Nouns are modified much better.</strong></span></span></li>
<li><span style="color: #ff0000;"><span style="color: #008000;"><strong>Although accuracy is lowwe than the first modification, the performance is better because we have tagged more words correctly.</strong></span></span></li>
<li><span style="color: #ff0000;"><span style="color: #008000;"><strong>64 changes are listed down in the data frame that were incorrectly determined by the vanilla algorithm.</strong></span></span></li>
</ul>
</li>
</ul>
<h1><span style="color: #ff0000;">Viterbi_test is the best algorithm</span></h1>

In [858]:
dt7[dt7['vanilla_tag']!=dt7['mod2_tag']]

,word,vanilla_tag,mod2_tag
0,Android,.,NOUN
1,Android,.,NOUN
2,Android,.,NOUN
3,Android,.,NOUN
41,Google,.,DET
42,Google,.,NOUN
43,Google,.,X
44,Google,.,DET
45,Google,.,NOUN
46,Google,.,X
